In [22]:
import numpy as np

In [23]:
def swipe_rows(A, first_row_index, second_row_index):
    A_copy = A.copy()
    first_row, second_row = A_copy[first_row_index, ], A_copy[second_row_index, ]
    A[first_row_index, ], A[second_row_index, ] = second_row, first_row

    return A

In [24]:
def increment_row(A, row_to_change, base_row, value):
    A_copy = A.copy()
    vector_to_change, base_vector = A_copy[row_to_change, ], A_copy[base_row, ]
    vector_changed = vector_to_change + base_vector * value
    A[row_to_change, ] = vector_changed

    return A

In [25]:
def multiply_row_by_scalar(A, row_to_change, value):
    A_copy = A.copy()
    vector_to_change = A_copy[row_to_change, ]
    vector_changed = vector_to_change * value
    A[row_to_change, ] = vector_changed

    return A

In [26]:
def divide_row_by_value(A, row_to_change, value):
    A_copy = A.copy()
    vector_to_change = A_copy[row_to_change, ]
    vector_changed = vector_to_change / value
    A[row_to_change, ] = vector_changed

    return A

In [27]:
def find_first_row_with_non_zero_element_at_index(A, start_row, column):
    A_size = A.shape[0]

    for row in range(start_row, A_size):
        if not np.isclose(A[row, column], 0):
            return row
    
    return -1


In [28]:
def find_last_non_null_row(A):
    A_size = A.shape[0]

    for row in range(A_size-1, 0, -1):
        if A[row,].any():
            return row
    
    return -1


In [29]:
def find_first_non_null_row(A, start_row):
    A_size = A.shape[0]

    for row in range(start_row, A_size):
        if A[row,].any():
            return row
    
    return -1


In [30]:
def find_first_non_zero_column(A):
    A_size = A.shape[1]

    for column in range(A_size):
        if A[:, column].any():
            return column
    
    return -1


In [31]:
def gauss_jordan_elimination(A, rows, columns, start_col):

    for row in range(rows):

        first_valid_row = find_first_non_null_row(A, row)

        if row != first_valid_row and first_valid_row != -1:
            swipe_rows(A, row, first_valid_row)
        
        A_copy = A.copy()

        while True:
            first_valid_column = find_first_row_with_non_zero_element_at_index(A, row, start_col)

            if first_valid_column != -1 or start_col + 1 == columns:
                break
                
            start_col += 1
        
        # It's not possible to follow because there isn't more valid rows or columns
        if first_valid_column == -1:
            break
        
        pivot = A_copy[row, start_col]

        for el in range(row+1, rows):
            element = A_copy[el, start_col]

            if element != 0:
                value = element / pivot

                if element + pivot * value != 0:
                    increment_row(A, el, row, -value)
                else:
                    increment_row(A, el, row, value)

        start_col += 1

    if first_valid_column != -1: 
        # decreases one because the last interation add an aditional count
        start_col -= 1

        for back_row in range(row, -1, -1):        
            A_copy = A.copy()
            
            pivot = A_copy[back_row, start_col]

            for el in range(back_row-1, -1, -1):
                element = A_copy[el, start_col]

                if not np.isclose(element, 0):
                    value = element / pivot

                    if not np.isclose(element + pivot * value, 0):
                        increment_row(A, el, back_row, -value)
                    else:
                        increment_row(A, el, back_row, value)
            
            value = 1 / pivot
            multiply_row_by_scalar(A, back_row, value)

            start_col -= 1


In [32]:
X = np.array([
    [3, -1, 4, 2, 5],
    [-2, 6, 1, -3, 4],
    [5, 2, -2, 1, -1],
    [1, 3, -4, 2, 6],
    [4, -2, 1, 5, -3]
], dtype=np.float64)

y = np.array([12, 7, 3, 8, 10], dtype=np.float64)

A = np.column_stack((X, y))

A

array([[ 3., -1.,  4.,  2.,  5., 12.],
       [-2.,  6.,  1., -3.,  4.,  7.],
       [ 5.,  2., -2.,  1., -1.,  3.],
       [ 1.,  3., -4.,  2.,  6.,  8.],
       [ 4., -2.,  1.,  5., -3., 10.]])

In [33]:
rows, columns = A.shape

current_col = find_first_non_zero_column(A)

if current_col == -1:
    print("Is needed at least one valid column")
else:
    row = 0
    while True:
        current_row = find_first_row_with_non_zero_element_at_index(A, row, current_col)

        if current_row != -1 or row+1 == rows:
            break
            
        row += 1

    if current_row != -1:
        gauss_jordan_elimination(A, rows, columns, current_col)
    else:
        print("Is needed at least one valid row and column")

In [36]:
print("RREF Matrix:")
print(A)

RREF Matrix:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -3.53177051e-02]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.98226403e+00]
 [-0.00000000e+00  4.48609550e-17  1.00000000e+00 -0.00000000e+00
  -0.00000000e+00  1.55721777e+00]
 [ 0.00000000e+00 -4.62307552e-17  9.24615104e-17  1.00000000e+00
   0.00000000e+00  2.78454658e+00]
 [-0.00000000e+00 -3.10944635e-17  6.21889270e-17 -0.00000000e+00
   1.00000000e+00  4.58050586e-01]]


In [35]:
y_hat = A[:, -1]

if np.allclose(X @ y_hat, y):
    print("Matrix is correct")
else:
    print("Matrix is incorrect")

Matrix is correct
